In [1]:
def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length
    
    

In [2]:

import os

config1_file = os.path.join("lj_sample_configurations", "lj_sample_config_periodic1.txt")

sample_coords, box_length = read_xyz(config1_file)

print(box_length)

print(len(sample_coords))

10.0
800


In [3]:
import math

def calculate_tail_correction(n,v,r_c):
    r3_term = math.pow(1/r_c, 3)
    r9_term = (1/3) * (math.pow(r3_term, 9))
    NV_term = (8*math.pi/3) * ((n**2)/v)
    tail_correction = NV_term * (r9_term - r3_term)
    
    return tail_correction
    

In [16]:
U = calculate_tail_correction(len(sample_coords), math.pow(box_length
,3),3)

In [17]:
#Write an assert statement to check against the reported value from NIST. 
#The value you are looking for is ULRC. We were using Configuration 1 in class.

assert math.isclose(U, -1.9849E+02, rel_tol=0.05)


In [22]:
def calculate_LJ(r_ij):
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    pairwise_energy = 4 * (r12_term - r6_term)
    return pairwise_energy
   


def calculate_distance(coord1, coord2):
    
    distance = 0
    
    for i in range(3):
        dim_dist = (coord1[i] - coord2[i]) ** 2
        distance += dim_dist
        
    distance = math.sqrt(distance)
    
    return distance

def calculate_total_energy(coordinates):
    total_energy = 0
    num_atoms = len(coordinates)
    
    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            dist_ij = calculate_distance(coordinates[i], coordinates[j])
            pairwise_energy = calculate_LJ(dist_ij)
            
            total_energy += pairwise_energy
    
    return total_energy

coordinates = [[0, 0, 0], [0, math.pow(2, 1/6), 0], [0, 2*math.pow(2, 1/6), 0]]

calculate_total_energy(coordinates)

-2.031005859375

In [31]:
#Answer this question - What is the difference in calculated system energy when there is no cut-off vs when using a cut-off of $3\sigma$?

def calculate_total_energy(coordinates, cutoff):
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_distance(coordinates[i], coordinates[j])

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
                
    return total_energy



In [33]:
calculate_total_energy(sample_coords, 3)

-3487.454232861954

In [34]:
calculate_total_energy(sample_coords, 1000)

-3582.239311831134

In [35]:
Cutoff vs. non cutoff
print(calculate_total_energy(sample_coords, 1000) - calculate_total_energy(sample_coords, 3))

-94.78507896917972


The difference between the cutoff value and the non cutoff value are that it excludes the energy where the atoms exceed 3 sigma Lennard Jones distances. This is reflected in the total energy difference between the cutoff and non cutoff. 